In [1]:
%pylab inline 
import pandas as pd 
import json

Populating the interactive namespace from numpy and matplotlib


In [2]:
# csv files for answer responses
gdrive_path = "/Users/dorislee/Google Drive/Turn/VAST/user_study/evaluation_study/transcription_and_analysis/"
# user study logs
box_path = "/Users/dorislee/Box/viz-summarization-user-study-material/" 

In [3]:
import glob

In [4]:
df_data = []
for filename in glob.glob(box_path+"*log"):
    line_num=0
    with open(filename) as f:
        for line in f:
            lsp = line.split(',')
            date = lsp[0]
            user = lsp[1][1:-1]
            task = lsp[2][1:-1]
            dataset_name = lsp[3][1:-1]
            clicks = json.loads(",".join(lsp[4:]))
            if dataset_name!="default":
                dataset = json.loads(open("../system/static/generated_dashboards/"+dataset_name).read().replace("\\",""))
                for nodeID in dataset.keys():
                    df_data.append([line_num,date,dataset_name,user,task,nodeID,clicks[nodeID],dataset[nodeID][-1]["filter"],dataset[nodeID][-1]["populationSize"]])
            line_num +=1

In [ ]:
1,13,4,8

In [66]:
df[df["visualization"].str.contains("duration=30\+ min")&~df["visualization"].str.contains(",")]#&(df["interested"]="interested")]

,user,visualization,interested,task
372,1,duration=30+ min,interested,A1
4480,10,duration=30+ min,not interested,A2
5852,12,duration=30+ min,unselected,A1
6522,13,duration=30+ min,interested,A1
7032,14,duration=30+ min,not interested,A1
7660,15,duration=30+ min,not interested,A2
8440,16,duration=30+ min,not interested,A2
742,2,duration=30+ min,unselected,A1
940,3,duration=30+ min,unselected,A2
1620,4,duration=30+ min,interested,A2


In [5]:
df = pd.DataFrame(df_data,columns=["line_num","date","dataset_name","user","task","nodeID","clicks","visualization","populationSize"])
df["date"] = pd.to_datetime(df["date"],format="%Y-%m-%d-%H-%M-%S")

In [6]:
df.loc[df["clicks"]==0,"interested"]="unselected"
df.loc[df["clicks"]==1,"interested"]="unselected"
df.loc[df["clicks"]==2,"interested"]="interested"
df.loc[df["clicks"]==3,"interested"]="not interested"

In [7]:
#select the best task, user (since all node are logged on the latest output, so we will just pick the latest one)
#df = df.loc[df.groupby(["user","task","nodeID"])[["date"]].idxmax()]
#select the latest logged output based on line numbering
df = df.loc[df.groupby(["user","task","nodeID"])["line_num"].idxmax()]

In [8]:
df = df[["user","visualization","interested","task"]]

In [9]:
def cleanFilterVal(x):
    if x=="#":
        return "overall"
    else:
        return x.replace("$","=").replace("#",", ")[1:-2]

In [10]:
df["visualization"]= df["visualization"].apply(cleanFilterVal)

In [11]:
df["user"] = df["user"].astype("int")

In [12]:
df = df[~df["task"].str.contains("T")] #ignore the titanic tutorial tasks

In [13]:
#make sure each user only does the task that they are assigned
user_task =  pd.read_csv(gdrive_path+"UserTaskAssignment.csv")

In [14]:
df.head()

,user,visualization,interested,task
376,1,overall,unselected,A1
378,1,"driver_race=Other, duration=30+ min",interested,A1
370,1,contraband_found=t,interested,A1
372,1,duration=30+ min,interested,A1
373,1,"contraband_found=t, driver_age=60 and over",unselected,A1


In [15]:
i =0 
df_data=[]
for row in df.iterrows():
    selected_user = user_task[user_task["User"]==row[1]["user"]]
    if row[1]["task"] in selected_user["Task1"].values[0] or row[1]["task"] in selected_user["Task2"].values[0]: 
        #print "pass"
        df_data.append(row[1][["user","visualization","interested","task"]])
    else:
        pass
        #print row[1]
    i +=1

In [16]:
df_data =pd.DataFrame(df_data)

- Had to manually clean up user_log_03_22.log because the second trial of user 11 was recorded as user 1, user 12 was also recorded as user 1
- Had to manually clean up user_log_03_18.log because user 8 was not recorded
- Had to manually clean up user_log_03_18.log because user  18 was recorded as user 1 for the second trial and there were A1 stuff mixed in there

cross checking that the results are same as what we expect

In [17]:
df_data.groupby(["user","task"],as_index=False).sum()[["user","task"]].groupby("user",as_index=False).sum()

,user,task
0,1,A1B2
1,2,A1B3
2,3,A2B1
3,4,A2B3
4,5,A3B1
5,6,A3B2
6,7,A1B2
7,8,A1B3
8,9,A2B1
9,10,A2B3


In [18]:
user_task

,User,Task1,Task2
0,1,A1,B2
1,2,A1,B3
2,3,A2,B1
3,4,A2,B3
4,5,A3,B1
5,6,A3,B2
6,7,A1,B2
7,8,A1,B3
8,9,A2,B1
9,10,A2,B3


In [19]:
df.to_csv("user_click_data.csv",index=None)

In [67]:
#cross check with screen shot 
# df[df["user"]==9]